In [1]:
import matplotlib.pyplot as plt
import json
import pprint
import requests
from skimage import io

%matplotlib inline

In [2]:
# Using Google Natural Language API to extract information from the questions
# the following file was generated locally as Google Natural Language API can't be used in cloud notebook

!curl https://www.dropbox.com/s/xcgkoohl3643mqo/train_100_keyphrases.json?dl=0 -L -o train_100_keyphrases.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4352  100  4352    0     0   4352      0  0:00:01  0:00:01 --:--:--     0


In [3]:
import pandas as pd
train_100_keyphrases = pd.read_json('train_100_keyphrases.json')
train_100_keyphrases['keyphrases'][0]

['name', 'product']

In [4]:
question_keyphrases = train_100_keyphrases['keyphrases']

In [5]:
# Using Microsoft Azure Computer Vision API to extract image features

subscription_key = "my key"  # my key
assert subscription_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze?"

In [6]:
# evaluate an image using the Microsoft Azure Cognitive Service Computer Vision API
def analyze_image(image_url):    
    headers = {'Ocp-Apim-Subscription-Key': subscription_key}
    params = {'visualFeatures': 'Adult,Categories,Description,Color,Faces,ImageType,Tags'}
    data = {'url': image_url}
    
    response = requests.post(vision_analyze_url, headers = headers, params = params, json = data)
    response.raise_for_status()
    analysis = response.json()
    return analysis

In [7]:
# Use the Optical Character Recognition (OCR) method to detect text in an image, 
# and then extract recognized characters into a machine-usable character stream.

ocr_url = vision_base_url + "ocr"
def analyze_image_text(image_url):   
    headers  = {'Ocp-Apim-Subscription-Key': subscription_key}
    params   = {'language': 'unk', 'detectOrientation ': 'true'}
    data     = {'url': image_url}
    
    response = requests.post(ocr_url, headers=headers, params=params, json=data)
    response.raise_for_status()
    analysis = response.json()
    
    line_infos = [region["lines"] for region in analysis["regions"]]
    word_infos = []
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                word_infos.append(word_info)
    word_infos
    return word_infos

In [8]:
# get a list of image urls 

image_urllist = []
index = ["%.2d" % i for i in range(100)]
for i in index: 
    image_url = 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/' + 'VizWiz_train_0000000000' + str(i) + '.jpg'
    image_urllist.append(image_url)

In [9]:
# extract image features of the given list, both image tags anad texts detected in the image
image_rawtags = []
image_rawtexts = []
for i in image_urllist:
    image_tag = analyze_image(i)
    image_rawtags.append(image_tag['tags'])

    image_text = analyze_image_text(i)
    image_rawtexts.append(image_text)

image_tags = []
for i in image_rawtags:
    image_tags.append([d['name'] for d in i if 'name' in d])

image_texts = []
for i in image_rawtexts:
    image_texts.append([d['text'] for d in i if 'text' in d])

In [10]:
image_texts[0]

['B', 'sil', 'Leaves', 'OZ', '(170)']

In [11]:
from textblob import TextBlob
joined_toblob = ' '.join(image_texts[0])
ngrams = TextBlob(joined_toblob).ngrams(n=2)
pprint.pprint(ngrams)

joined_list = []
for i in ngrams:
    joined = ' '.join(i)
    joined_list.append(joined)
pprint.pprint(joined_list)

[WordList(['B', 'sil']),
 WordList(['sil', 'Leaves']),
 WordList(['Leaves', 'OZ']),
 WordList(['OZ', '170'])]
['B sil', 'sil Leaves', 'Leaves OZ', 'OZ 170']


In [12]:
from autocorrect import spell
corrected_list = []
for i in joined_list:
    corrected = spell(i)
    corrected_list.append(corrected)
print(corrected_list)

['Basil', 'sil Leaves', 'Leaves OZ', 'OZ 170']


In [13]:
# ['B', 'sil', 'Leaves', 'OZ', '(170)']
# remove the non-words in corrected_list and put the corrected_list to image_text
import enchant
d = enchant.Dict("en_US")
corrected = []
for i in corrected_list:
    if d.check(i) == True:
        corrected.append(i)
print(corrected)

['Basil']


In [14]:
# Above shows the detailed procesure, now get a list of corrected words extracted from the OCR, 
# and append them to image_texts list

import enchant
d = enchant.Dict("en_US")
corrected_ngrams = []
for i in image_texts:
    joined_toblob = ' '.join(i)
    ngram = TextBlob(joined_toblob).ngrams(n=2)
    corrected = []
    for k in ngram:
        joined_k = ' '.join(k)
        corrected_k = spell(joined_k)
        if d.check(corrected_k) == True:
            corrected.append(corrected_k)
    corrected_ngrams.append(corrected)

#pprint.pprint(corrected_ngrams)

In [15]:
train_features = []
for a,b,c in zip(image_tags, image_texts, corrected_ngrams):
    train_features.append(a+b+c)

train_features[0]

['bottle', 'indoor', 'B', 'sil', 'Leaves', 'OZ', '(170)', 'Basil']

In [16]:
# get the majority vote of the 10 answers

import pandas as pd
df_train = pd.read_json('/Users/xuxinyi/Downloads/ML/train.json')
rawanswers = df_train['answers']
answers = []
for i in rawanswers:
    answers.append([d['answer'] for d in i])
    
pprint.pprint(answers[0])

['basil leaves',
 'basil leaves',
 'basil',
 'basil',
 'basil leaves',
 'basil leaves',
 'basil leaves',
 'basil leaves',
 'basil leaves',
 'basil']


In [17]:
def most_common(lst):
    return max(set(lst), key=lst.count)

major_answers = []
for i in answers:
    major_answers.append(most_common(i))
    
pprint.pprint(major_answers[0])

'basil leaves'


In [18]:
# using Keras, tensorflow to do the question answering task
# ajusted from https://github.com/sujitpal/dl-models-for-qa/blob/master/src/qa-lstm-cnn.py

# -*- coding: utf-8 -*-
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Merge, Flatten, Dropout, concatenate
from keras.layers import Conv1D, GlobalMaxPool1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import numpy as np
import os

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
## extract training data, get train question answer pairs
train_qapairs = []
a = []
# b = []
questions = df_train['question']

for question in questions:   
    qwords = nltk.word_tokenize(question)
    a.append([qwords])

# for answer in major_answers:
#     awords = nltk.word_tokenize(answer)
#     b.append([awords])


for i,j in zip(a,train_features):
    train_qapairs.append(i+[j])
    
print(train_qapairs[0])

[['What', "'s", 'the', 'name', 'of', 'this', 'product', '?'], ['bottle', 'indoor', 'B', 'sil', 'Leaves', 'OZ', '(170)', 'Basil']]


In [21]:
def build_vocab(stories, qapairs, testqs):
    wordcounts = collections.Counter()
    for story in stories:
        for sword in story:
            wordcounts[sword] += 1
    for qapair in qapairs:
        for qword in qapair[0]:
            wordcounts[qword] += 1
        for feature in qapair[1]:
            wordcounts[feature] += 1
    for testq in testqs:
        for qword in testq[0]:
            wordcounts[qword] += 1
        for feature in testq[1]:
            wordcounts[feature] += 1
    words = [wordcount[0] for wordcount in wordcounts.most_common()]
    word2idx = {w: i+1 for i, w in enumerate(words)}  # 0 = mask
    return word2idx

# Pads sequences to the same length. This function transforms a list of num_samples sequences (lists of integers) 
# into a 2D Numpy array of shape  (num_samples, num_timesteps). num_timesteps is either the maxlen argument 
# if provided, or the length of the longest sequence otherwise.
# Sequences that are shorter than num_timesteps are padded with value at the end.

from keras.preprocessing.sequence import pad_sequences
def vectorize_qapairs(qapairs, word2idx, seq_maxlen):
    Xq, Xf = [], []
    for qapair in qapairs:
        Xq.append([word2idx[qword] for qword in qapair[0]])
        Xf.append([word2idx[feature] for feature in qapair[1]])
    return (pad_sequences(Xq, maxlen=seq_maxlen),
            pad_sequences(Xf, maxlen=seq_maxlen))

In [22]:
## extract data
import collections
import keras
from keras.utils import to_categorical

question_maxlen = max([len(train_qapair[0]) for train_qapair in train_qapairs])
answer_maxlen = max([len(train_qapair[1]) for train_qapair in train_qapairs])
seq_maxlen = max([question_maxlen, answer_maxlen])

word2idx = build_vocab([], train_qapairs, [])   
vocab_size = len(word2idx) + 1 # include mask character 0

Xq_train, Xf_train = vectorize_qapairs(train_qapairs, word2idx, seq_maxlen)

# Convert labels to categorical one-hot encoding
y_train = df_train['answerable'][:100]
one_hot_labels = keras.utils.to_categorical(y_train)

print(Xf_train.shape)
print(vocab_size)
print(seq_maxlen)

(100, 278)
715
278


In [23]:
Xq_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [24]:
Xf_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [25]:
import operator
sorted_word2idx = sorted(word2idx.items(), key=operator.itemgetter(1))
sorted_word2idx

[('?', 1),
 ('this', 2),
 ('is', 3),
 ('What', 4),
 ('indoor', 5),
 ('the', 6),
 ('of', 7),
 ('you', 8),
 ('in', 9),
 ('.', 10),
 ('what', 11),
 ('can', 12),
 ('to', 13),
 ('please', 14),
 ('and', 15),
 ('person', 16),
 ('I', 17),
 (',', 18),
 ('that', 19),
 ('a', 20),
 ('me', 21),
 ('on', 22),
 ('kind', 23),
 ('are', 24),
 ('/', 25),
 ('bottle', 26),
 ('Can', 27),
 ('tell', 28),
 ('for', 29),
 ('color', 30),
 ('floor', 31),
 ('name', 32),
 ('wall', 33),
 ('these', 34),
 ('close', 35),
 ('it', 36),
 ('Is', 37),
 ('or', 38),
 ('Thank', 39),
 ('hand', 40),
 ('computer', 41),
 ('does', 42),
 ('beverage', 43),
 ('Vitamin', 44),
 ('my', 45),
 ('v', 46),
 ('was', 47),
 ("'s", 48),
 ('electronics', 49),
 ('drink', 50),
 ('flavor', 51),
 ('blur', 52),
 ('table', 53),
 ('desk', 54),
 ('say', 55),
 ('see', 56),
 ('picture', 57),
 ('outdoor', 58),
 ('A', 59),
 ('tree', 60),
 ('Of', 61),
 ('not', 62),
 ('product', 63),
 ('up', 64),
 ('•', 65),
 ('This', 66),
 ('look', 67),
 ('food', 68),
 ('How', 

In [26]:
# get embeddings from word2vec

from gensim.models import KeyedVectors

WORD2VEC_EMBED_SIZE = 300 
# why 300? According to research, the quality for vector representations improves as you increase the vector size 
# until you reach 300 dimensions.  http://www.aclweb.org/anthology/D14-1162

word2vec = KeyedVectors.load_word2vec_format(
    '/Users/xuxinyi/Downloads/ML/GoogleNews-vectors-negative300.bin.gz', binary=True)  
embedding_weights = np.zeros((vocab_size, WORD2VEC_EMBED_SIZE))
for word, index in word2idx.items():
    try:
        embedding_weights[index, :] = word2vec[word.lower()]
    except KeyError:
        pass  # keep as zero (not ideal, but what else can we do?)
    
print(embedding_weights)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.109375    0.140625   -0.03173828 ...  0.00765991  0.12011719
  -0.1796875 ]
 ...
 [-0.04125977 -0.20800781  0.06445312 ... -0.18457031  0.17871094
  -0.00747681]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.08203125  0.31640625  0.12792969 ... -0.18847656  0.10546875
   0.359375  ]]


In [27]:
embedding_weights.shape
# why later it runs error? 
# ValueError: Layer weight shape (162, 300) not compatible with provided weight shape (716, 300)


(715, 300)

In [28]:
# build model, add LSTM and CNN layers

from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Merge, Flatten, Dropout, concatenate
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import text, sequence


QA_EMBED_SIZE = 128
qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   input_length=seq_maxlen,
                   weights=[embedding_weights]))     ### notice embedding_weights
qenc.add(LSTM(QA_EMBED_SIZE, return_sequences=True))
qenc.add(Dropout(0.3))
qenc.add(Conv1D(QA_EMBED_SIZE, kernel_size = 5, padding='valid'))# the quotient in which the digits after the decimal point are removed
qenc.add(MaxPooling1D(pool_size=2, padding='valid'))
qenc.add(Dropout(0.3))
qenc.add(Flatten())
print(qenc.summary())

fenc = Sequential()
fenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   input_length=seq_maxlen,
                   weights=[embedding_weights]))
fenc.add(LSTM(QA_EMBED_SIZE, return_sequences=True))
fenc.add(Dropout(0.3))
fenc.add(Conv1D(QA_EMBED_SIZE, kernel_size = 5, padding='valid'))
fenc.add(MaxPooling1D(pool_size=2, padding='valid'))
fenc.add(Dropout(0.3))
fenc.add(Flatten())
print(fenc.summary())

model = Sequential()
# keras.layers.Concatenate(axis=-1)
model.add(Merge([qenc, fenc], mode="concat", concat_axis=-1))
model.add(Dense(2, activation="sigmoid")) 
# change activation from softmax to relu, the loss is suddenly decrease and the accuracy is no longer 0.656

model.compile(optimizer='adam', loss="categorical_crossentropy",    # loss="categorical_crossentropy"  error, see description
              metrics=["accuracy"])                            # optimizer='rmsprop' < optimizer="adam"
print(model.summary())

### Use 'softmax' and 'categorical_crossentropy' instead of 'sigmoid' and 'binary_crossentropy'. 
### By using the latter two, you're treating the problem as a multilabel problem, not a multiclass problem.

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 278, 300)          214500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 278, 128)          219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 278, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 274, 128)          82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 137, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 137, 128)          0         
___________________________________________________________

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [29]:
# checkpoint = ModelCheckpoint(
#     filepath=os.path.join(MODEL_DIR, "qa-lstm-cnn-best.hdf5"),
#     verbose=1, save_best_only=True)
BATCH_SIZE = 32
NBR_EPOCHS = 50
model.fit([Xq_train, Xf_train], one_hot_labels, batch_size=BATCH_SIZE,    ### notice fit both Xqtrain and Xatrain
          epochs=NBR_EPOCHS) 

Epoch 1/50
100/100 [==============================] - 4s 40ms/step - loss: 0.6724 - acc: 0.6100
Epoch 2/50
100/100 [==============================] - 2s 24ms/step - loss: 0.6608 - acc: 0.6500
Epoch 3/50
100/100 [==============================] - 2s 23ms/step - loss: 0.6556 - acc: 0.6500
Epoch 4/50
100/100 [==============================] - 2s 23ms/step - loss: 0.6724 - acc: 0.6500
Epoch 5/50
100/100 [==============================] - 2s 23ms/step - loss: 0.6578 - acc: 0.6500
Epoch 6/50
100/100 [==============================] - 2s 23ms/step - loss: 0.6438 - acc: 0.6500
Epoch 7/50
100/100 [==============================] - 2s 23ms/step - loss: 0.6559 - acc: 0.6500
Epoch 8/50
100/100 [==============================] - 2s 24ms/step - loss: 0.6332 - acc: 0.6500
Epoch 9/50
100/100 [==============================] - 2s 25ms/step - loss: 0.6638 - acc: 0.6600
Epoch 10/50
100/100 [==============================] - 3s 28ms/step - loss: 0.6727 - acc: 0.6600
Epoch 11/50
100/100 [==================

In [30]:
# prepare the X_test data
image_urllist = []
index = ["%.2d" % i for i in range(30)]
for i in index: 
    image_url = 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/' + 'VizWiz_test_0000000200' + str(i) + '.jpg'
    image_urllist.append(image_url)

# extract image features of the given list, both image tags anad texts detected in the image
image_rawtags = []
image_rawtexts = []
for i in image_urllist:
    image_tag = analyze_image(i)
    image_rawtags.append(image_tag['tags'])

    image_text = analyze_image_text(i)
    image_rawtexts.append(image_text)

image_tags = []
for i in image_rawtags:
    image_tags.append([d['name'] for d in i if 'name' in d])

image_texts = []
for i in image_rawtexts:
    image_texts.append([d['text'] for d in i if 'text' in d])
    
from textblob import TextBlob
joined_toblob = ' '.join(image_texts[0])
ngrams = TextBlob(joined_toblob).ngrams(n=2)

joined_list = []
for i in ngrams:
    joined = ' '.join(i)
    joined_list.append(joined)

from autocorrect import spell
corrected_list = []
for i in joined_list:
    corrected = spell(i)
    corrected_list.append(corrected)
import enchant
d = enchant.Dict("en_US")
corrected = []
for i in corrected_list:
    if d.check(i) == True:
        corrected.append(i)

import enchant
d = enchant.Dict("en_US")
corrected_ngrams = []
for i in image_texts:
    joined_toblob = ' '.join(i)
    ngram = TextBlob(joined_toblob).ngrams(n=2)
    corrected = []
    for k in ngram:
        joined_k = ' '.join(k)
        corrected_k = spell(joined_k)
        if d.check(corrected_k) == True:
            corrected.append(corrected_k)
    corrected_ngrams.append(corrected)
    
test_features = []
for a,b,c in zip(image_tags, image_texts, corrected_ngrams):
    test_features.append(a+b+c)

## extract training data, get train question answer pairs
df_test = pd.read_json('/Users/xuxinyi/Downloads/ML/test.json')

testqs = []
a = []
questions = df_test['question']

for question in questions:   
    qwords = nltk.word_tokenize(question)
    a.append([qwords])

for i,j in zip(a,test_features):
    testqs.append(i+[j])

In [34]:
# question_maxlen = max([len(testqs[0]) for testq in testqs])
# answer_maxlen = max([len(test_qapair[1]) for test_qapair in test_qapairs])
# seq_maxlen = question_maxlen

word2idx = build_vocab([],[], testqs)   
vocab_size = len(word2idx) + 1 # include mask character 0

seq_maxlen =278
def vectorize_testqs(testqs, word2idx, seq_maxlen):
    Xq, Xf = [], []
    for testq in testqs:
        Xq.append([word2idx[qword] for qword in testq[0]])
        Xf.append([word2idx[feature] for feature in testq[1]])
    return (pad_sequences(Xq, maxlen=seq_maxlen),
            pad_sequences(Xf, maxlen=seq_maxlen))

Xq_test, Xf_test = vectorize_testqs(testqs, word2idx, seq_maxlen)

print(Xf_test.shape)
print(seq_maxlen)

(30, 278)
278


In [35]:
# print("Evaluation...")
# loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
# print("Test loss/accuracy final model = %.4f, %.4f" % (loss, acc))

# model.save_weights(os.path.join(MODEL_DIR, "qa-lstm-attn-final.hdf5"))
# with open(os.path.join(MODEL_DIR, "qa-lstm-attn.json"), "wb") as fjson:
#     fjson.write(model.to_json())

# model.load_weights(filepath=os.path.join(MODEL_DIR, 
#                                          "qa-lstm-attn-best.hdf5"))
# loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
# print("Test loss/accuracy best model = %.4f, %.4f" % (loss, acc))

In [36]:
y_pred = model.predict([Xq_test, Xf_test], batch_size = BATCH_SIZE)
print(y_pred)
# sample_submission = pd.read_csv("../sample_submission.csv")
# sample_submission['answerable'] = y_test
# sample_submission.to_csv("predictions.csv", index=False)

[[8.3643175e-04 9.9808758e-01]
 [7.7132356e-01 8.2058437e-02]
 [1.5007910e-08 1.0000000e+00]
 [9.4667518e-01 1.5575873e-02]
 [4.4975374e-02 8.7884325e-01]
 [9.8537016e-01 4.4306829e-03]
 [7.3010311e-02 8.2579619e-01]
 [5.6172784e-02 8.4647238e-01]
 [4.3406408e-02 8.8014656e-01]
 [7.7138585e-01 8.5565098e-02]
 [8.1453407e-01 6.2369730e-02]
 [3.6071464e-02 8.9638603e-01]
 [5.5535417e-02 8.4816688e-01]
 [3.1519797e-01 4.1519490e-01]
 [2.3502968e-01 5.1171148e-01]
 [9.9977559e-01 5.6821475e-05]
 [2.3827709e-03 9.9372971e-01]
 [4.8970247e-05 9.9988270e-01]
 [9.3680501e-07 9.9999845e-01]
 [3.9457764e-02 8.8896614e-01]
 [2.8829006e-03 9.9285114e-01]
 [9.1051632e-01 2.7507091e-02]
 [7.5852475e-04 9.9852604e-01]
 [9.9437124e-01 1.5875258e-03]
 [9.9191898e-01 2.1879070e-03]
 [9.2385424e-09 1.0000000e+00]
 [3.4978703e-04 9.9918133e-01]
 [2.8292960e-02 9.2531145e-01]
 [4.2356560e-03 9.8897237e-01]
 [8.4776685e-02 7.7939260e-01]]


In [39]:
pprint.pprint(y_pred.tolist())

[[0.000836431747302413, 0.9980875849723816],
 [0.771323561668396, 0.08205843716859818],
 [1.5007909581754575e-08, 1.0],
 [0.946675181388855, 0.015575872734189034],
 [0.04497537389397621, 0.8788432478904724],
 [0.9853701591491699, 0.004430682864040136],
 [0.07301031053066254, 0.8257961869239807],
 [0.05617278441786766, 0.8464723825454712],
 [0.0434064082801342, 0.8801465630531311],
 [0.7713858485221863, 0.08556509763002396],
 [0.814534068107605, 0.062369730323553085],
 [0.03607146441936493, 0.8963860273361206],
 [0.055535417050123215, 0.8481668829917908],
 [0.31519797444343567, 0.41519489884376526],
 [0.2350296825170517, 0.5117114782333374],
 [0.9997755885124207, 5.682147457264364e-05],
 [0.0023827708791941404, 0.9937297105789185],
 [4.8970247007673606e-05, 0.999882698059082],
 [9.368050086777657e-07, 0.9999984502792358],
 [0.039457764476537704, 0.8889661431312561],
 [0.002882900647819042, 0.9928511381149292],
 [0.9105163216590881, 0.02750709094107151],
 [0.000758524751290679, 0.9985260

In [42]:
pred_answerable = []
for i in y_pred:
    pred = i[1]
    if pred > 0.5:
        pred = 1
    else:
        pred = 0
    pred_answerable.append(pred)

pprint.pprint(pred_answerable)

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1]
